Подключаем нужные пакеты для работы с БД и данными

In [2]:
import pandas as pd
import sqlite3 as sql

Создадим базу данных с таблицей по данной схеме

```mermaid
# Здесь будет схема
```


** schema.sql**
```SQL
CREATE TABLE orders(
    user_id  INT,              -- уникальный идентификатор пользователя
    order_id INT,              -- уникальный идентификатор заказа
    order_time INT,            -- время сделанного заказа в unixtime (секунды, UTC)
    order_cost REAL,           -- стоимость заказа
    success_order_flag BOOLEAN -- идентификатор, определяющий был ли заказ 
);
```
db = sqlite3.connect(path)

In [3]:
columns = ["user_id","order_id","order_time","order_cost","success_order_flg"]
db = sql.connect('../../../files/db.sqlite')
c = db.cursor()

Заполним таблицу [данными](https://drive.google.com/file/d/1SY9wK0EWaBCr9R3cLbqhtW4l7FQPq4Gr/view?usp=sharing) из csv файла

In [4]:
path = 'dataset.csv'
users = pd.read_csv(path,delimiter=';')
# преобразуем int в дату
users['order_time'] = pd.to_datetime(users['order_time'],unit='s')
users.to_sql('orders', db, index = False)

ValueError: Table 'orders' already exists.

Проверим что данные в базе данных 😃

In [ ]:
table = c.execute("""
    SELECT * FROM orders LIMIT 10
""").fetchall()
display(pd.DataFrame(table,columns=columns))

,user_id,order_id,order_time,order_cost,success_order_flg
0,1728898,118881751,2020-01-06 21:14:09,12804.1667,0
1,1728898,132150404,2020-03-16 14:03:17,5147.2728,1
2,1728898,133654929,2020-03-29 17:00:00,8600.0,1
3,1728898,133770763,2020-03-26 15:06:09,6807.3031,1
4,1766832,118492969,2020-01-09 15:09:26,2625.0,1
5,1766832,120888914,2020-01-18 10:33:02,2408.0,0
6,1766832,124602594,2020-02-09 11:53:48,7494.0,1
7,1766832,124604579,2020-02-12 09:13:41,2913.3332,1
8,1766832,125072398,2020-02-09 19:59:45,1765.5001,1
9,1766832,125198531,2020-02-10 10:48:12,1536.6668,1


Нужно получить отчетную таблицу
 - date - календарная дата привлечения или реактивации
 - gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
 - gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date
 - users_count_new - кол-во пользователей, привлеченных в день date
 - users_count_reactivated- кол-во пользователей, реактивированных в день date


In [31]:
user_id = '43166214'
c.executescript(open("views.sql").read())
table = c.execute(f'SELECT * FROM Result')
df = pd.DataFrame(table,columns=['date','new sum','renew sum','sum','new count','renew count','count'])
display(len(df))
display(df.head(10))
display(df.tail(10))


150

,date,new sum,renew sum,sum,count_new,count_renew,count
0,2020-01-01,3.464499e+05,0.0,3.464499e+05,12,0,12
1,2020-01-02,3.705565e+06,0.0,3.705565e+06,60,0,60
2,2020-01-03,8.383269e+06,0.0,8.383269e+06,171,0,171
3,2020-01-04,1.157345e+07,0.0,1.157345e+07,265,0,265
4,2020-01-05,9.226659e+06,0.0,9.226659e+06,232,0,232
5,2020-01-06,1.014439e+07,0.0,1.014439e+07,290,0,290
6,2020-01-07,3.522883e+06,0.0,3.522883e+06,148,0,148
7,2020-01-08,8.598805e+06,0.0,8.598805e+06,313,0,313
8,2020-01-09,1.075165e+07,0.0,1.075165e+07,411,0,411
9,2020-01-10,1.035178e+07,0.0,1.035178e+07,377,0,377


,date,new sum,renew sum,sum,count_new,count_renew,count
140,2020-05-22,8525.9088,165117.5154,173643.4242,2,12,14
141,2020-05-23,5953.3334,73252.1813,79205.5147,1,11,12
142,2020-05-24,19767.3333,78236.3631,98003.6964,2,12,14
143,2020-05-25,40695.9998,88847.2118,129543.2116,1,22,23
144,2020-05-26,3980.0000,87802.0160,91782.0160,1,14,15
145,2020-05-27,0.0000,89709.5910,89709.5910,0,16,16
146,2020-05-28,5896.0000,89389.5303,95285.5303,2,21,23
147,2020-05-29,10903.1210,136378.8633,147281.9843,2,16,18
148,2020-05-30,0.0000,73687.7273,73687.7273,0,17,17
149,2020-05-31,4280.4242,166821.5758,171102.0000,2,23,25


In [33]:
import plotly.graph_objects as go
 
plot = go.Figure(data=[go.Bar(
    name = 'Новые',
    x = df['date'],
    y = df['new sum']
   ),
                       go.Bar(
    name = 'Реактивированные',
    x = df['date'],
    y = df['renew sum']
   )
])
plot.update_layout(barmode='stack')
plot.show()
#
plot = go.Figure(data=[go.Bar(
    name = 'Новые',
    x = df['date'],
    y = df['new count']
   ),
                       go.Bar(
    name = 'Реактивированные',
    x = df['date'],
    y = df['renew count']
   )
])
plot.update_layout(barmode='stack')
plot.show()

KeyError: 'new count'